# Finetune models

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets
!pip install codecarbon
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

## Import necessary libraries

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import torch
from torch import nn
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          AutoModel,
                          BertTokenizer,
                          BertModel,
                          DistilBertModel,
                          RobertaModel)
from huggingface_hub import (
    PyTorchModelHubMixin,
    notebook_login,
    ModelCard,
    ModelCardData,
    EvalResult,
)
from datasets import DatasetDict, load_dataset
from torch.utils.data import Dataset, DataLoader

from codecarbon import (
    EmissionsTracker,
    track_emissions,
)



In [2]:
notebook_login(new_session=False)

## Create custom dataset class

In [3]:
# Defining the TextDataset class

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors="pt",
        )
        self.labels = torch.tensor([int(l[0]) for l in labels])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self) -> int:
        return len(self.labels)

In [93]:
import nltk
from nltk.stem import WordNetLemmatizer
import contractions
import re

nltk.download('punkt_tab')
nltk.download('wordnet')

def preprocess(X):
    lemmatizer  = WordNetLemmatizer()
    preprocessed_texts = []
    for doc in X:
        # Expand contractions
        expanded = contractions.fix(doc)
        # Remove special characters
        # expanded = re.sub(r"[^a-zA-Z0-9]", " ", expanded)
        # Lowercase
        lowered = expanded.lower()
        # Tokenize and lemmatize
        lemmatized = " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(lowered)])
        preprocessed_texts.append(lemmatized)
    return preprocessed_texts

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/terryzhang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/terryzhang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


!## Model set up

In [94]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [95]:
class BertClassifier(nn.Module, PyTorchModelHubMixin):
    def __init__(self, num_labels=8,
                 bert_variety="bert-base-uncased",
                 dropout_rate=0.05):
        super().__init__()
        self.bert_variety = bert_variety
        if bert_variety.startswith("distil"):
            self.bert = DistilBertModel.from_pretrained(bert_variety)
            self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        elif bert_variety.startswith("bert"):
            self.bert = BertModel.from_pretrained(bert_variety)
            self.classifier = nn.Linear(self.bert.pooler.dense.out_features, num_labels)

        self.config = self.bert.config
        self.config.num_labels = num_labels
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        if self.bert_variety.startswith("distil"):
            # Use first token hidden state ([CLS] token)
            pooled_output = outputs.last_hidden_state[:, 0, :]
        else:
            pooled_output = outputs.pooler_output  # Already pooled [CLS] with tanh

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits
        return logits

class BertFreezeClassifier(nn.Module, PyTorchModelHubMixin):
    """
    A BERT-based classifier with frozen layers.
    This model is designed to freeze the embedding layer and the first 10 transformer layers.
    """
    def __init__(self, num_labels=8,
                 bert_variety="bert-base-uncased",
                 dropout_rate=0.05,
                 freeze_layers=10):
        super().__init__()
        self.bert_variety = bert_variety
        if bert_variety.startswith("distil"):
            self.bert = DistilBertModel.from_pretrained(bert_variety)
            self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        elif bert_variety.startswith("bert"):
            self.bert = BertModel.from_pretrained(bert_variety)
            self.classifier = nn.Linear(self.bert.pooler.dense.out_features, num_labels)

        self.config = self.bert.config
        self.config.num_labels = num_labels

        # Freeze embedding layer and first 10 transformer layers
        self.freeze_model_layers(freeze_layers)

        self.dropout = nn.Dropout(dropout_rate)

    def freeze_model_layers(self, freeze_layers):
        if self.bert_variety.startswith("distil"):
            layer_names = ("embeddings", *(f"transformer.layer.{i}" for i in range(freeze_layers)))
        else:
            layer_names = ("embeddings", *(f"encoder.layer.{i}" for i in range(freeze_layers)))

        for name, param in self.bert.named_parameters():
            if name.startswith(layer_names):
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        if self.bert_variety.startswith("distil"):
            # Use first token hidden state ([CLS] token)
            pooled_output = outputs.last_hidden_state[:, 0, :]
        else:
            pooled_output = outputs.pooler_output  # Already pooled [CLS] with tanh

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

class TfidfXGBClassifier():
    def __init__(self, num_labels=8):
        self.xgb_pipeline = Pipeline([
            # TF-IDF vectorization
            ("tfidf", TfidfVectorizer(max_features=2000, min_df=2, ngram_range=(1, 3))),
            ("classifier", XGBClassifier(objective="multi:softmax",
                                        eval_metric='merror',
                                        num_class=num_labels,
                                        random_state=42))  # XGBoost
        ])

    def fit(self, X, y):
        self.xgb_pipeline.fit(X, y)
        return self

    def predict(self, X):
        return self.xgb_pipeline.predict(X)


class RobertaClassifier(nn.Module, PyTorchModelHubMixin):
    def __init__(self, num_labels=8, model_name="roberta-base", dropout_rate=0.05):
        super(RobertaClassifier, self).__init__()
        self.model_name = model_name

        if model_name.startswith("distilroberta"):
            self.model = DistilBertModel.from_pretrained(model_name)
            self.classifier = nn.Linear(self.model.config.hidden_size, num_labels)
        elif model_name.startswith("roberta"):
            self.model = RobertaModel.from_pretrained(model_name)
            self.classifier = nn.Linear(self.model.config.hidden_size, num_labels)
        else:
            raise ValueError("Model name should start with 'roberta' or 'distilroberta'.")

        self.config = self.model.config
        self.config.num_labels = num_labels
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        if self.model_name.startswith("distilroberta"):
            pooled_output = outputs.last_hidden_state[:, 0, :]
        else:
            pooled_output = outputs.pooler_output

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [104]:
class ConspiracyClassifier768(
    nn.Module,
    PyTorchModelHubMixin
):
    def __init__(self, num_classes=8):
        super().__init__()
        self.h1 = nn.Linear(768, 512)
        self.bn1 = nn.BatchNorm1d(512)

        self.h2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)

        self.h3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)

        self.h4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)

        self.h5 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.2)
        self.activation = nn.ReLU()

        self.initialize_weights()


    def forward(self, input_texts):
        outputs = self.h1(input_texts)
        outputs = self.bn1(outputs)
        outputs = self.activation(outputs)
        outputs = self.h2(outputs)
        outputs = self.bn2(outputs)
        outputs = self.activation(outputs)
        outputs = self.h3(outputs)
        outputs = self.bn3(outputs)
        outputs = self.activation(outputs)
        outputs = self.dropout(outputs)
        outputs = self.h4(outputs)
        outputs = self.bn4(outputs)
        outputs = self.activation(outputs)
        outputs = self.h5(outputs)

        return outputs

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity="relu")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

### Train XGB

In [97]:
class TrainTfidfXGB():
    def __init__(self, model, dataset):
        self.model = model
        self.dataset = dataset
        self.train_texts, self.train_labels, self.val_texts, self.val_labels = self.prepare_dataset_tfidf()

    def prepare_dataset_tfidf(self):
        print("Loading dataset...")
        # Load the dataset
        dataset = self.dataset

        # Split dataset
        train_texts = dataset["train"]["quote"]
        train_labels = [int(l[0]) for l in dataset["train"]["label"]]
        val_texts = dataset["test"]["quote"]
        val_labels = [int(l[0]) for l in dataset["test"]["label"]]

        train_texts_preprocessed = preprocess(train_texts)
        val_texts_preprocessed = preprocess(val_texts)

        print("Dataset loaded and dataloaders created.")
        return train_texts_preprocessed, train_labels, val_texts_preprocessed, val_labels

    def train(self):
        print("Training XGBoost model...")
        self.model.fit(self.train_texts, self.train_labels)
        print("XGBoost model trained.")


    def evaluate(self):
        print("Evaluating XGBoost model...")
        predictions = self.model.predict(self.val_texts)
        accuracy = accuracy_score(self.val_labels, predictions)
        f1 = f1_score(self.val_labels, predictions, average="weighted")
        print(f"Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")
        return accuracy, f1


In [13]:
test_train_tfidf_xgb = TrainTfidfXGB(model=TfidfXGBClassifier(), dataset=load_dataset("quotaclimat/frugalaichallenge-text-train"))

README.md:   0%|          | 0.00/5.80k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/248k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1219 [00:00<?, ? examples/s]

Loading dataset...
Dataset loaded and dataloaders created.


In [108]:
class TrainBERT:
    def __init__(self, model, tokenizer, dataset, batch_size=32, epochs=3, test=False):
        self.model = model
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer)
        self.embedding_model = AutoModel.from_pretrained(tokenizer)
        self.dataset = dataset
        self.batch_size = batch_size
        self.epochs = epochs
        self.training_progress = []
        self.test = test

        self.train_dataloader, self.val_dataloader = self.prepare_dataset_bert()

        if torch.backends.mps.is_available():
            self.device = torch.device("mps")
            torch.mps.empty_cache()
        elif torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")

        self.model.to(self.device)
        self.embedding_model.to(self.device)


    def prepare_dataset_bert(self):
        print("Loading dataset...")
        # Load the dataset
        dataset = self.dataset

        # Split dataset
        train_texts = dataset["train"]["quote"]
        train_labels = dataset["train"]["label"]
        val_texts = dataset["test"]["quote"]
        val_labels = dataset["test"]["label"]


        # Create the datasets
        train_dataset = TextDataset(train_texts, train_labels, self.tokenizer)
        val_dataset = TextDataset(val_texts, val_labels, self.tokenizer)

        # Create the dataloaders
        train_dataloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False)

        print("Dataset loaded and dataloaders created.")
        return train_dataloader, val_dataloader


    def custom_print_time(self, additional_text):
        """
        Custom function to print the current time with a message.
        """
        time_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"{time_str} {additional_text}")


    def model_metrics(self, model, dataloader):
        criterion = nn.CrossEntropyLoss()
        model.eval()
        with torch.no_grad():
            total_loss = 0
            total_correct = 0
            total_length = 0
            for batch in dataloader:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)

                if isinstance(self.model, ConspiracyClassifier768):
                    embeddings = self.get_embeddings(input_ids, attention_mask, self.embedding_model)
                    outputs = self.model(embeddings)
                else:
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

                loss = criterion(outputs, labels)

                predicted = torch.argmax(outputs, 1).cpu().numpy()
                labels = labels.cpu().numpy()

                # Calculate accuracy
                total_correct += (predicted == labels).sum().item()
                total_length += len(labels)
                total_loss += loss.item()

            average_loss = total_loss / len(dataloader)
            average_acc = total_correct / total_length

        model.train()
        return average_loss, average_acc

    def eval_model_status(self, epoch):
        train_loss, train_acc = self.model_metrics(self.model, self.train_dataloader)
        test_loss, test_acc = self.model_metrics(self.model, self.val_dataloader)
        loss_str = f"Loss: Train {train_loss:0.3f}, Test {test_loss:0.3f}"
        acc_str = f"Acc: Train {train_acc:0.3f}, Test {test_acc:0.3f}"
        self.custom_print_time(f"Epoch {epoch+1:2}/{self.epochs} done. {loss_str}; and {acc_str}")
        metrics = dict(
            epoch=epoch,
            train_loss=train_loss,
            train_acc=train_acc,
            test_loss=test_loss,
            test_acc=test_acc,
        )
        return metrics

    def get_embeddings(self, input_ids, attention_mask, embedding_model):
        with torch.no_grad():
            embeddings = embedding_model(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        return embeddings

    def train_model(self, lr=2e-5):
        # Define the optimizer
        optimizer = torch.optim.AdamW(self.model.parameters(), lr)
        criterion = nn.CrossEntropyLoss()

        self.model.train()

        print("Training the model...")
        # Training loop
        if not self.test:
            for epoch in range(self.epochs):
                total_loss = 0
                self.custom_print_time(f"Epoch {epoch + 1}/{self.epochs}")
                for batch in self.train_dataloader:
                    input_ids = batch["input_ids"].to(self.device)
                    attention_mask = batch["attention_mask"].to(self.device)
                    labels = batch["labels"].to(self.device)

                    if isinstance(self.model, ConspiracyClassifier768):
                        embeddings = self.get_embeddings(input_ids, attention_mask, self.embedding_model)
                        outputs = self.model(embeddings)
                    else:
                        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

                    loss = criterion(outputs, labels)

                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                eval_metrics = self.eval_model_status(epoch)
                self.training_progress.append(eval_metrics)
                total_loss += loss.item()

        else:
            print("Test mode enabled, skipping training.")
            eval_metrics = self.eval_model_status(0)
            self.training_progress.append(eval_metrics)

        print("Training complete.")

In [109]:
test_train_bert = TrainBERT(ConspiracyClassifier768(), "sentence-transformers/all-distilroberta-v1", load_dataset("quotaclimat/frugalaichallenge-text-train"), batch_size=32, epochs=3)

Loading dataset...
Dataset loaded and dataloaders created.


In [110]:
test_train_bert.train_model(lr=2e-5)

Training the model...
2025-04-11 14:25:00 Epoch 1/3
2025-04-11 14:25:59 Epoch  1/3 done. Loss: Train 1.719, Test 1.792; and Acc: Train 0.360, Test 0.331
2025-04-11 14:25:59 Epoch 2/3
2025-04-11 14:26:59 Epoch  2/3 done. Loss: Train 1.487, Test 1.585; and Acc: Train 0.469, Test 0.420
2025-04-11 14:26:59 Epoch 3/3
2025-04-11 14:27:59 Epoch  3/3 done. Loss: Train 1.335, Test 1.447; and Acc: Train 0.545, Test 0.481
Training complete.


In [105]:
models = {
    # "tfidf-xgboost": {
    #     "name": "TFIDF + XGBoost",
    #     "tokenizer": "tfidf",
    #     "model": TfidfXGBClassifier()
    # },
    # "bert-base-uncased": {
    #     "name": "BERT Base Uncased",
    #     "tokenizer": "bert-base-uncased",
    #     "model": BertClassifier(bert_variety="bert-base-uncased"),
    # },
    # "bert-base-uncased-freeze": {
    #     "name": "BERT Base Uncased with Frozen Layers",
    #     "tokenizer": "bert-base-uncased",
    #     "model": BertFreezeClassifier(bert_variety="bert-base-uncased", freeze_layers=10),
    # },
    # "distilbert-base-uncased": {
    #     "name": "DistilBERT",
    #     "tokenizer": "distilbert-base-uncased",
    #     "model": BertClassifier(bert_variety="distilbert-base-uncased"),
    # },
    # "roberta-base": {
    #     "name": "RoBERTa Base",
    #     "tokenizer": "roberta-base",
    #     "model": RobertaClassifier(model_name="roberta-base"),
    # },
    # "distilroberta-base": {
    #     "name": "DistilRoBERTa Base",
    #     "tokenizer": "distilroberta-base",
    #     "model": RobertaClassifier(model_name="distilroberta-base"),
    # },
    # "sentence-bert": {
    #     "name": "Sentence-BERT",
    #     "tokenizer": "sentence-transformers/all-MiniLM-L6-v2",
    #     "model": "sentence-transformers/all-MiniLM-L6-v2"
    # },
    "conspiracy-768": {
        "name": "custom MLP 768 model",
        "tokenizer": "sentence-transformers/all-distilroberta-v1",
        "model": ConspiracyClassifier768(),
    },
}

In [15]:
# train_base_bert = TrainBERT(
#     model=models["bert-base-uncased"]["model"],
#     tokenizer=models["bert-base-uncased"]["tokenizer"],
#     dataset="quotaclimat/frugalaichallenge-text-train",
#     batch_size=32,
#     epochs=3
# )

# train_base_bert.train_model(lr=2e-5)

## Measure carbon

In [ ]:
from codecarbon import EmissionsTracker
import time

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate_model_performance(model, device, dataloader):
    """
    Evaluate the performance of a trained model on a given dataloader.

    Args:
        model: Trained PyTorch model.
        dataloader: DataLoader containing the evaluation dataset.

    Returns:
        int: Accuracy of the model on the evaluation dataset.
        int: F1 score of the model on the evaluation dataset.
    """
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs, dim=1).cpu().numpy()
            all_predictions.extend(predictions)
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions, average="weighted")

    return accuracy, f1

In [ ]:
# Run models and measure carbon footprint
def train_eval_model_with_tracking(model_config, dataset, batch_size=32, epochs=3, test_mode=False):
    """
    Train and evaluate a model while tracking its carbon footprint.

    Args:
        model: The model to be trained.
        tokenizer: The tokenizer to be used with the model.
        dataset: The dataset to be used for training and evaluation.
        batch_size: The batch size for training.
        epochs: The number of epochs for training.
        test_mode: If True, the model will not be trained, only evaluated.

    Returns:
        dict: A dictionary containing the model, energy consumed, carbon emissions, hardware used, training time, accuracy, and F1 score.
    """

    # Train NN models
    if 'bert' in model_config['name'].lower():
        training_run = TrainBERT(
            model=model_config["model"],
            tokenizer=model_config["tokenizer"],
            dataset=dataset,
            batch_size=batch_size,
            epochs=epochs,
            test=test_mode
        )

        start_time = time.time()
        tracker = EmissionsTracker(project_name="FrugalAI_model_carbon",
                                measure_power_secs=10)
        tracker.start()

        training_run.train_model(lr=2e-5)

        end_time = time.time()
        emissions = tracker.stop()

        accuracy, f1 = evaluate_model_performance(
                training_run.model, training_run.device, training_run.val_dataloader
            )

    elif 'tfidf' in model_config['name'].lower():
        training_run = TrainTfidfXGB(
            model=model_config["model"],
            dataset=dataset,
        )
        start_time = time.time()
        tracker = EmissionsTracker(project_name="FrugalAI_model_carbon",
                                measure_power_secs=10)
        tracker.start()

        training_run.train()


        end_time = time.time()
        emissions = tracker.stop()

        # Evaluate the model
        accuracy, f1 = training_run.evaluate()

        batch_size = 'N/A'
        epochs = 'N/A'

    results = {
        "model_name": model_config['name'],
        "batch_size": batch_size,
        "epochs": epochs,
        'location': tracker._geo.country_iso_code,
        "energy_consumed_kwh": tracker._total_energy.kWh,
        "carbon_emissions_kgco2": emissions,
        'cpu_energy_kwh': tracker._total_cpu_energy.kWh,
        'gpu_energy_kwh': tracker._total_gpu_energy.kWh,
        'ram_energy_kwh': tracker._total_ram_energy.kWh,
        'hardware': tracker._hardware,
        "training_time": end_time - start_time,
        'accuracy': accuracy,
        'f1_score': f1,
    }


    return training_run.model, results

In [ ]:
# model_config = models['tfidf-xgboost']
# dataset = load_dataset("quotaclimat/frugalaichallenge-text-train")

# model, results = train_eval_model_with_tracking(
#         model_config=model_config,
#         dataset=dataset,
#         batch_size=32,
#         epochs=3,
#         test_mode=True
#     )

Loading dataset...


[codecarbon INFO @ 18:58:40] [setup] RAM Tracking...
[codecarbon INFO @ 18:58:40] [setup] CPU Tracking...
[codecarbon WARNING @ 18:58:40] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU



Dataset loaded and dataloaders created.


[codecarbon WARNING @ 18:58:41] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 18:58:41] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 18:58:41] [setup] GPU Tracking...
[codecarbon INFO @ 18:58:41] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:58:41] >>> Tracker's metadata:
[codecarbon INFO @ 18:58:41]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 18:58:41]   Python version: 3.11.11
[codecarbon INFO @ 18:58:41]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 18:58:41]   Available RAM : 83.477 GB
[codecarbon INFO @ 18:58:41]   CPU count: 12
[codecarbon INFO @ 18:58:41]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 18:58:41]   GPU count: 1
[codecarbon INFO @ 18:58:41]   GPU model: 1 x NVIDIA A100-SXM4-40GB
[codecarbon INFO @ 18:58:42] Saving emissions data to file /content/emissions.csv


Training XGBoost model...


[codecarbon INFO @ 18:58:52] Energy consumed for RAM : 0.000087 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 18:58:52] Energy consumed for all CPUs : 0.000118 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:58:52] Energy consumed for all GPUs : 0.000142 kWh. Total GPU Power : 51.188462033740144 W
[codecarbon INFO @ 18:58:52] 0.000347 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:59:02] Energy consumed for RAM : 0.000174 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 18:59:02] Energy consumed for all CPUs : 0.000236 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:59:02] Energy consumed for all GPUs : 0.000286 kWh. Total GPU Power : 51.6520364384468 W
[codecarbon INFO @ 18:59:02] 0.000696 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:59:05] Energy consumed for RAM : 0.000203 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 18:59:05] Energy consumed for all CPUs : 0.000276 kWh. Total CPU Power : 42.5 W
[codecarbo

XGBoost model trained.
Evaluating XGBoost model...
Accuracy: 0.5849, F1 Score: 0.5785


/usr/local/lib/python3.11/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


In [ ]:
trained_models = {}
results_list = []

dataset = load_dataset("quotaclimat/frugalaichallenge-text-train")

for model_name, model_config in models.items():
    print(f"Training {model_name}")

    model, results = train_eval_model_with_tracking(
        model_config=model_config,
        dataset=dataset,
        batch_size=32,
        epochs=10,
    )

    trained_models[model_name] = model
    results_list.append(results)
    print(f"Results for {model_name}: {results}")

Training roberta-base


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

2025-04-08 22:35:05 Epoch  8/10 done. Loss: Train 0.086, Test 2.109; and Acc: Train 0.973, Test 0.543
2025-04-08 22:35:05 Epoch 9/10


[codecarbon INFO @ 22:35:10] Energy consumed for RAM : 0.002173 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 22:35:10] Energy consumed for all CPUs : 0.002950 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:35:10] Energy consumed for all GPUs : 0.024076 kWh. Total GPU Power : 365.00285542757416 W
[codecarbon INFO @ 22:35:10] 0.029200 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:35:20] Energy consumed for RAM : 0.002260 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 22:35:20] Energy consumed for all CPUs : 0.003068 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:35:20] Energy consumed for all GPUs : 0.024988 kWh. Total GPU Power : 328.3675403969384 W
[codecarbon INFO @ 22:35:20] 0.030316 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:35:30] Energy consumed for RAM : 0.002347 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 22:35:30] Energy consumed for all CPUs : 0.003187 kWh. Total CPU Power : 42.5 W
[codecarb

2025-04-08 22:35:36 Epoch  9/10 done. Loss: Train 0.029, Test 2.134; and Acc: Train 0.995, Test 0.559
2025-04-08 22:35:36 Epoch 10/10


[codecarbon INFO @ 22:35:40] Energy consumed for RAM : 0.002434 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 22:35:40] Energy consumed for all CPUs : 0.003305 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:35:40] Energy consumed for all GPUs : 0.026977 kWh. Total GPU Power : 365.4105662899322 W
[codecarbon INFO @ 22:35:40] 0.032715 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:35:50] Energy consumed for RAM : 0.002521 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 22:35:50] Energy consumed for all CPUs : 0.003423 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:35:50] Energy consumed for all GPUs : 0.027898 kWh. Total GPU Power : 331.6270167562139 W
[codecarbon INFO @ 22:35:50] 0.033841 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:36:00] Energy consumed for RAM : 0.002607 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 22:36:00] Energy consumed for all CPUs : 0.003541 kWh. Total CPU Power : 42.5 W
[codecarbo

2025-04-08 22:36:07 Epoch 10/10 done. Loss: Train 0.014, Test 2.161; and Acc: Train 0.998, Test 0.576
Training complete.
Results for distilroberta-base: {'model_name': 'DistilRoBERTa Base', 'batch_size': 32, 'epochs': 10, 'location': 'SGP', 'energy_consumed_kwh': 0.0358222374509346, 'carbon_emissions_kgco2': 0.016864500413863344, 'cpu_energy_kwh': 0.0036145016011437706, 'gpu_energy_kwh': 0.029545785858831997, 'ram_energy_kwh': 0.002661949990958832, 'hardware': [RAM(), CPU(Intel(R) Xeon(R) CPU @ 2.20GHz > 85W [generic]), GPU() (NVIDIA A100-SXM4-40GB)], 'training_time': 307.76089096069336, 'accuracy': 0.5758818703855619, 'f1_score': 0.5764872141594377}


In [ ]:
# prompt: write the code to help pickle the models in a dictionary based on the name as the filename and the time

import pickle
import os
from datetime import datetime

def pickle_models(models, results_list):
  """Pickles trained models and saves results.

  Args:
    models: A dictionary of trained models.
    results_list: A list of dictionaries containing model results.
  """

  # Create a directory to store the pickled models and results if it doesn't exist
  timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
  output_dir = f"/content/drive/MyDrive/Colab Notebooks/FrugalAI/saved_models/pickled_models_{timestamp}"
  os.makedirs(output_dir, exist_ok=True)


  for model_name, trained_model in models.items():
    filename = os.path.join(output_dir, f"{model_name}.pkl")
    try:
      with open(filename, "wb") as file:
        pickle.dump(trained_model, file)
      print(f"Model '{model_name}' pickled successfully to '{filename}'")
    except Exception as e:
      print(f"Error pickling model '{model_name}': {e}")

  # Save results to a file
  results_filename = os.path.join(output_dir, "results.pkl")
  try:
    with open(results_filename, "wb") as file:
      pickle.dump(results_list, file)
    print(f"Results pickled successfully to '{results_filename}'")
  except Exception as e:
      print(f"Error pickling results: {e}")


# Example usage (assuming 'models' and 'results_list' are defined)
pickle_models(trained_models, results_list)


Model 'roberta-base' pickled successfully to '/content/drive/MyDrive/Colab Notebooks/FrugalAI/saved_models/pickled_models_20250408224521/roberta-base.pkl'
Model 'distilroberta-base' pickled successfully to '/content/drive/MyDrive/Colab Notebooks/FrugalAI/saved_models/pickled_models_20250408224521/distilroberta-base.pkl'
Error pickling results: ctypes objects containing pointers cannot be pickled


In [ ]:
pd.DataFrame(results_list).to_csv("/content/drive/MyDrive/Colab Notebooks/FrugalAI/results_v2.csv")

In [ ]:
# Train model
model = TrainBERT(
    model=models["distilbert-base-uncased"]["model"],
    tokenizer=models["distilbert-base-uncased"]["tokenizer"],
    dataset="quotaclimat/frugalaichallenge-text-train",
    batch_size=32,
    epochs=1,
    test=True
)

start_time = time.time()
tracker = EmissionsTracker(project_name="FrugalAI_model_carbon",
                           measure_power_secs=10,)
tracker.start()

model.train_model(lr=2e-5)

end_time = time.time()
emissions = tracker.stop()

accuracy, f1 = evaluate_model_performance(
        model.model, model.device, model.val_dataloader
    )


results = {
    "model": model,
    "energy_consumed_kwh": tracker._total_energy.kwh,
    "carbon_emissions_kgco2": emissions,
    'hardware': tracker._hardware,
    "training_time": end_time - start_time,
    'accuracy': accuracy,
    'f1_score': f1,
}

# energy_consumed = tracker.final_emissions_data["energy_consumed"]
# carbon_emissions = tracker.final_emissions_data["carbon_emissions"]


Loading dataset...
Dataset loaded and dataloaders created.


TypeError: __init__() got an unexpected keyword argument 'experiment_id'

In [4]:
dataset = load_dataset("quotaclimat/frugalaichallenge-text-train")

Using the latest cached version of the dataset since quotaclimat/frugalaichallenge-text-train couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/terryzhang/.cache/huggingface/datasets/quotaclimat___frugalaichallenge-text-train/default/0.0.0/4748e7b548154d188a377a96e104b9b31fcd45b4 (last modified on Sat Mar  1 17:00:14 2025).


In [5]:
# Convert the dataset to a pandas DataFrame
train_df = dataset['train'].to_pandas()
test_df = dataset['test'].to_pandas()

# Generate summary statistics for the train and test datasets
train_summary = train_df.describe(include='all')
test_summary = test_df.describe(include='all')

# Display the summary statistics
print("Train Dataset Summary:")
print(train_summary)

print("\nTest Dataset Summary:")
print(test_summary)

Train Dataset Summary:
                                                    quote           label  \
count                                                4872            4872   
unique                                               4872               8   
top     Interesting to note that Oklahoma minimum temp...  0_not_relevant   
freq                                                    1            1311   
mean                                                  NaN             NaN   
std                                                   NaN             NaN   
min                                                   NaN             NaN   
25%                                                   NaN             NaN   
50%                                                   NaN             NaN   
75%                                                   NaN             NaN   
max                                                   NaN             NaN   

        source                                      

In [9]:
import pandas as pd

In [ ]:
combi_df['word_count'] = combi_df['quote'].apply(lambda row: len(row.split()))

In [14]:
combi_df = pd.concat([train_df, test_df])[['quote', 'label']]

In [24]:
combi_df['length'] = combi_df['quote'].apply(lambda row: len(row.split()))

In [45]:
combi_df.describe()

,length
count,6091.000000
mean,48.854704
std,42.517348
min,3.000000
25%,24.000000
50%,38.000000
75%,61.000000
max,789.000000


In [44]:
# combi_df.groupby('label').agg({'label':'count', 'length':['min', 'mean', 'max']}).round(0)

total_row = combi_df.groupby('label').agg({'label':'count', 'length':['min', 'mean', 'max']}).round(0).sum(numeric_only=True)
total_row.name = 'Total'
data_summary_df = combi_df.groupby('label').agg({'label':'count', 'length':['min', 'mean', 'max']}).round(0)
pd.concat([data_summary_df, pd.DataFrame(total_row)], axis=1)

,"(label, count)","(length, min)","(length, mean)","(length, max)",Total
0_not_relevant,1618.0,4.0,48.0,630.0,NaN
1_not_happening,741.0,4.0,37.0,399.0,NaN
2_not_human,702.0,5.0,50.0,404.0,NaN
3_not_bad,386.0,3.0,45.0,374.0,NaN
4_solutions_harmful_unnecessary,774.0,10.0,55.0,378.0,NaN
5_science_unreliable,801.0,5.0,54.0,789.0,NaN
6_proponents_biased,782.0,5.0,48.0,454.0,NaN
7_fossil_fuels_needed,287.0,8.0,56.0,329.0,NaN
"(label, count)",NaN,NaN,NaN,NaN,6091.0
"(length, min)",NaN,NaN,NaN,NaN,44.0


AttributeError: 'DataFrame' object has no attribute 'append'